<a href="https://colab.research.google.com/github/priyanshgupta1998/All_codes/blob/master/intursion1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Signature based intrusion detection system

In [0]:

## Preliminaries
import numpy as np 
import matplotlib.pyplot as plt

import pandas as pd
import urllib
import csv


In [0]:
from sklearn import datasets, preprocessing, feature_extraction

In [0]:

from sklearn import linear_model, svm, metrics, ensemble, tree
from sklearn.decomposition import PCA

In [0]:
from sklearn import cross_validation

In [0]:
# Helper functions
def folds_to_split(data,targets,train,test):
    data_tr = pd.DataFrame(data).iloc[train]
    data_te = pd.DataFrame(data).iloc[test]
    labels_tr = pd.DataFrame(targets).iloc[train]
    labels_te = pd.DataFrame(targets).iloc[test]
    return [data_tr, data_te, labels_tr, labels_te]

In [0]:

#let's load the data
train_data = pd.read_csv('/home/kddcup.data_10_percent_corrected.csv')
test_data =  pd.read_csv('/home/corrected.csv')

In [0]:
#Place both dataset into a dataframe
train_multiclass = pd.read_csv('/home/kddcup.data_10_percent_corrected.csv', quotechar=',', skipinitialspace=True , names=['Duration', 'protocol_type', 'Service', 'Flag', 'src_bytes', 'dst_bytes', 'Land', 'wrong_fragment', 'Urgent', 'Hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'Count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate','Class'])

In [35]:
train_multiclass.shape

(283453, 42)

In [36]:
train_multiclass.head()

,Duration,protocol_type,Service,Flag,src_bytes,dst_bytes,Land,wrong_fragment,Urgent,Hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9.0,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19.0,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29.0,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39.0,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49.0,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [37]:
train_multiclass.tail()

,Duration,protocol_type,Service,Flag,src_bytes,dst_bytes,Land,wrong_fragment,Urgent,Hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
283448,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
283449,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
283450,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
283451,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
283452,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
test_multiclass = pd.read_csv('/home/corrected.csv', quotechar=',', skipinitialspace=True, names=['Duration', 'protocol_type', 'Service', 'Flag', 'src_bytes', 'dst_bytes', 'Land', 'wrong_fragment', 'Urgent', 'Hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'Count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate','Class'])

In [39]:
test_multiclass.shape

(303644, 42)

#Dos attack**

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='smurf.')|(train_multiclass['Class'] =='neptune.') | (train_multiclass['Class'] =='back.') | (train_multiclass['Class'] =='teardrop.') |(train_multiclass['Class'] =='pod.')| (train_multiclass['Class']=='land.'),'Class'] = 'DOS_ATTACK'

#Probe attack

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='satan.')|(train_multiclass['Class'] =='ipsweep.') | (train_multiclass['Class'] =='portsweep.') | (train_multiclass['Class'] =='nmap.'),'Class'] = 'probe'

#R2L Attack

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='spy.')|(train_multiclass['Class'] =='phf.')|(train_multiclass['Class'] =='multihop.')|(train_multiclass['Class'] =='ftp_write.') | (train_multiclass['Class'] =='imap.') | (train_multiclass['Class'] =='warezmaster.') |(train_multiclass['Class'] =='guess_passwd.')| (train_multiclass['Class']=='warezclient.'),'Class'] = 'r2l'

#U2R attack

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='buffer_overflow.')|(train_multiclass['Class'] =='rootkit.') | (train_multiclass['Class'] =='loadmodule.') | (train_multiclass['Class'] =='perl.'),'Class']='u2r'

#Normal attack

In [0]:
train_multiclass.loc[(train_multiclass['Class'] =='normal.'),'Class'] = 'normal'

`Now make a class for similar kind of attacks  IN test_set`

#Dos attack

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='smurf.')|(test_multiclass['Class'] =='neptune.') | 
                    (test_multiclass['Class'] =='back.') | (test_multiclass['Class'] =='teardrop.') |
                    (test_multiclass['Class'] =='pod.')| (test_multiclass['Class']=='land.')|
                   (test_multiclass['Class']=='apache2.')|(test_multiclass['Class']=='udpstorm.')|
                   (test_multiclass['Class']=='processtable.')|(test_multiclass['Class']=='mailbomb.'),'Class'] = 'Dos'


#r2l attack

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='guess_passwd.')|(test_multiclass['Class'] =='ftp_write.')|
                    (test_multiclass['Class'] =='imap.')|(test_multiclass['Class'] =='phf.') | 
                    (test_multiclass['Class'] =='multihop.') | 
                    (test_multiclass['Class'] =='warezmaster.') |(test_multiclass['Class'] =='snmpgetattack.')| 
                    (test_multiclass['Class']=='named.')|(test_multiclass['Class'] =='xlock.')|
                    (test_multiclass['Class'] =='xsnoop.')|(test_multiclass['Class'] =='sendmail.')|
                    (test_multiclass['Class'] =='httptunnel.')|(test_multiclass['Class'] =='worm.')|
                    (test_multiclass['Class'] =='snmpguess.'),'Class'] = 'r2l'

#probe attack

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='satan.')|(test_multiclass['Class'] =='ipsweep.') | (test_multiclass['Class'] =='portsweep.') | (test_multiclass['Class'] =='nmap.')|
                    (test_multiclass['Class'] =='saint.')|(test_multiclass['Class'] =='mscan.'),'Class'] = 'probe'


#u2r attack

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='buffer_overflow.')|(test_multiclass['Class'] =='rootkit.') | 
                    (test_multiclass['Class'] =='loadmodule.') | (test_multiclass['Class'] =='xterm.')|
                    (test_multiclass['Class'] =='sqlattack.')|(test_multiclass['Class'] =='ps.')|
                    (test_multiclass['Class'] =='perl.'),'Class']='u2r'

#normal attacks

In [0]:
test_multiclass.loc[(test_multiclass['Class'] =='normal.'),'Class'] = 'normal'

In [0]:
attr_encoder = feature_extraction.DictVectorizer(sparse=False)

In [54]:
attr_encoder

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)

In [0]:
label_encoder = preprocessing.LabelEncoder()

In [0]:
train_data_df_m = attr_encoder.fit_transform(train_multiclass.iloc[:,:-1].T.to_dict().values())

In [59]:
type(train_data_df_m)

numpy.ndarray

In [62]:
train_data_df_m[0:2]

array([[8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 5.45e+03,
        9.00e+00, 0.00e+00, 0.00e+00, 1.10e-01, 